An alternate idea, something along the lines of this one is to use the same divergence metric, but calculate it between a subject's roi (or all of their vertex) pairwise between the same roi from every other subject.

In an ML context, this would be calculate it between every subject in the training set, and then set as the mean.

In [1]:
import numpy as np
from neurotools.loading import load
from neurotools.transform.network import _load_fs_subj_data
from scipy.stats import ks_2samp
import os
from itertools import combinations

In [2]:
data_dr = 'data/ds003097/derivatives/freesurfer/'
subj_drs = [os.path.join(data_dr, s) for s in os.listdir(data_dr) if 'sub-' in s]
subj_drs = subj_drs[:10]

In [3]:
def compare_subjs(subj_dr1, subj_dr2, modality='thickness', parc='aparc.a2009s.annot'):
    
    data1, labels1 = _load_fs_subj_data(subj_dr1, modality, parc)
    data2, labels2 = _load_fs_subj_data(subj_dr2, modality, parc)
    
    difs = np.zeros(len(np.unique(labels1)))
    for i, u in enumerate(np.unique(labels1)):
        difs[i] = ks_2samp(data1[labels1==u], data2[labels2==u]).statistic
        
    return difs

In [4]:
def generate_differences(subjs, train_subjs):
    
    all_difs = []
    cache = {}
    
    for subj in subjs:
        print(subj)
        
        subj_difs = []
        for t_subj in train_subjs:
            
            print(t_subj)
            
            # Skip repeats
            if subj == t_subj:
                continue
            
            # If not already run
            if (subj, t_subj) not in cache:
                
                # Calc
                difs = compare_subjs(subj, t_subj)
                
                # Add to cache
                cache[(subj, t_subj)] = difs
                cache[(t_subj, subj)] = difs

            # Add
            subj_difs.append(cache[(subj, t_subj)])
        
        # Add mean across rois
        all_difs.append(np.mean(subj_difs, axis=0))
        
    return all_difs

In [5]:
all_difs = generate_differences(subj_drs, subj_drs)

data/ds003097/derivatives/freesurfer/sub-0265
data/ds003097/derivatives/freesurfer/sub-0265
data/ds003097/derivatives/freesurfer/sub-0856
data/ds003097/derivatives/freesurfer/sub-0208
data/ds003097/derivatives/freesurfer/sub-0856
data/ds003097/derivatives/freesurfer/sub-0265
data/ds003097/derivatives/freesurfer/sub-0856
data/ds003097/derivatives/freesurfer/sub-0208
data/ds003097/derivatives/freesurfer/sub-0208
data/ds003097/derivatives/freesurfer/sub-0265
data/ds003097/derivatives/freesurfer/sub-0856
data/ds003097/derivatives/freesurfer/sub-0208
